# Baseline Experiment
Extracting C++ optimizer and running a short optimization loop.

In [ ]:
import json
import os
import subprocess
import pandas as pd
import numpy as np
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
from shapely.strtree import STRtree

# Setup paths
KERNEL_PATH = '../../research/kernels/smartmanoj_santa-claude/santa-claude.ipynb'
DATA_DIR = '/home/data'
SUBMISSION_FILE = 'submission.csv'

# 1. Extract C++ code
print("Extracting C++ code...")
with open(KERNEL_PATH, 'r') as f:
    nb = json.load(f)

for cell in nb['cells']:
    if cell['cell_type'] == 'code':
        source = ''.join(cell['source'])
        if '%%writefile a.cpp' in source:
            content = source.replace('%%writefile a.cpp\n', '')
            with open('tree_packer.cpp', 'w') as f_out:
                f_out.write(content)
            print("Extracted tree_packer.cpp")
        if '%%writefile bp.cpp' in source:
            content = source.replace('%%writefile bp.cpp\n', '')
            with open('bp.cpp', 'w') as f_out:
                f_out.write(content)
            print("Extracted bp.cpp")

# 2. Compile C++ code
print("Compiling C++ code...")
subprocess.run(['g++', '-O3', '-march=native', '-std=c++17', '-fopenmp', '-o', 'tree_packer', 'tree_packer.cpp'], check=True)
subprocess.run(['g++', '-O3', '-std=c++17', '-o', 'bp', 'bp.cpp'], check=True)
print("Compilation successful.")

# 3. Initialize submission
print("Initializing submission...")
subprocess.run(['cp', os.path.join(DATA_DIR, 'sample_submission.csv'), SUBMISSION_FILE], check=True)

# 4. Optimization Loop (Short run for baseline)
print("Running optimization loop...")
# Run tree_packer
# -n 5000 iterations, -r 16 restarts (as per seed prompt for quick check)
subprocess.run(['./tree_packer', '-i', SUBMISSION_FILE, '-o', 'submission_opt.csv', '-n', '5000', '-r', '16'], check=True)

# Run Backward Propagation
subprocess.run(['./bp', 'submission_opt.csv', SUBMISSION_FILE], check=True)

# Run tree_packer again
subprocess.run(['./tree_packer', '-i', SUBMISSION_FILE, '-o', 'submission_opt.csv', '-n', '5000', '-r', '16'], check=True)

# Copy result back to submission.csv
subprocess.run(['cp', 'submission_opt.csv', SUBMISSION_FILE], check=True)
print("Optimization finished.")


In [ ]:
# Validation Code
getcontext().prec = 30
scale_factor = 1

class ChristmasTree:
    """Represents a single, rotatable Christmas tree of a fixed size."""

    def __init__(self, center_x='0', center_y='0', angle='0'):
        """Initializes the Christmas tree with a specific position and rotation."""
        self.center_x = Decimal(center_x)
        self.center_y = Decimal(center_y)
        self.angle = Decimal(angle)

        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon(
            [
                # Start at Tip
                (Decimal('0.0') * scale_factor, tip_y * scale_factor),
                # Right side - Top Tier
                (top_w / Decimal('2') * scale_factor, tier_1_y * scale_factor),
                (top_w / Decimal('4') * scale_factor, tier_1_y * scale_factor),
                # Right side - Middle Tier
                (mid_w / Decimal('2') * scale_factor, tier_2_y * scale_factor),
                (mid_w / Decimal('4') * scale_factor, tier_2_y * scale_factor),
                # Right side - Bottom Tier
                (base_w / Decimal('2') * scale_factor, base_y * scale_factor),
                # Right Trunk
                (trunk_w / Decimal('2') * scale_factor, base_y * scale_factor),
                (trunk_w / Decimal('2') * scale_factor, trunk_bottom_y * scale_factor),
                # Left Trunk
                (-(trunk_w / Decimal('2')) * scale_factor, trunk_bottom_y * scale_factor),
                (-(trunk_w / Decimal('2')) * scale_factor, base_y * scale_factor),
                # Left side - Bottom Tier
                (-(base_w / Decimal('2')) * scale_factor, base_y * scale_factor),
                # Left side - Middle Tier
                (-(mid_w / Decimal('4')) * scale_factor, tier_2_y * scale_factor),
                (-(mid_w / Decimal('2')) * scale_factor, tier_2_y * scale_factor),
                # Left side - Top Tier
                (-(top_w / Decimal('4')) * scale_factor, tier_1_y * scale_factor),
                (-(top_w / Decimal('2')) * scale_factor, tier_1_y * scale_factor),
            ]
        )
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated,
                                          xoff=float(self.center_x * scale_factor),
                                          yoff=float(self.center_y * scale_factor))

def load_configuration_from_df(n: int, df: pd.DataFrame) -> list[ChristmasTree]:
    group_data = df[df["id"].str.startswith(f"{n:03d}_")]
    trees = []
    for _, row in group_data.iterrows():
        x = str(row["x"])
        y = str(row["y"])
        deg = str(row["deg"])
        if x and y and deg:
            trees.append(ChristmasTree(x, y, deg))
    return trees

def get_score(trees: list[ChristmasTree], n: int) -> float:
    if not trees: return 0.0
    xys = np.concatenate([np.asarray(t.polygon.exterior.xy).T / float(scale_factor) for t in trees])
    min_x, min_y = xys.min(axis=0)
    max_x, max_y = xys.max(axis=0)
    side_length = max(max_x - min_x, max_y - min_y)
    return side_length**2 / n

def has_overlap(trees: list[ChristmasTree]) -> bool:
    if len(trees) <= 1: return False
    polygons = [t.polygon for t in trees]
    tree_index = STRtree(polygons)
    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx == i: continue
            if poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                return True
    return False

def score_and_validate_submission(file_path: str, max_n: int = 200) -> dict:
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        return {"status": "FAILED", "error": "File Not Found"}
    
    total_score = 0.0
    failed_overlap_n = []
    
    print(f"--- Scoring and Validation: {file_path} (N=1 to {max_n}) ---")
    for n in range(1, max_n + 1):
        trees = load_configuration_from_df(n, df)
        if trees:
            current_score = get_score(trees, n)
            total_score += current_score
            if has_overlap(trees):
                failed_overlap_n.append(n)
                print(f"  ❌ N={n:03d}: OVERLAP DETECTED! (Score contribution: {current_score:.12f})")
    
    if failed_overlap_n:
        status = "FAILED (Overlaps)"
    else:
        status = "SUCCESS"
        
    print(f"**Total Submission Score (Σ S²/N):   {total_score:.14f}**")
    return {"status": status, "total_score": total_score, "failed_overlap_n": failed_overlap_n}

# Validate
result = score_and_validate_submission(SUBMISSION_FILE)
print(f"Final Result: {result}")
